## Lecture 26: NLTK continued

__Reading material:__
- [NLTK Book](http://www.nltk.org/book/ch01.html) 1.3, 1.4, 3.1, 3.6, 5.1

Take a look at [this](https://www.theguardian.com/books/2009/apr/03/agatha-christie-alzheimers-research) article about declining metrics of language use in Agatha Christie’s later books. Is it possible to detect Alzheimer’s or other mental conditions from a person’s writing? We’re not going to answer that question today, but natural language processing is a very powerful and promising field!

1.Before you begin, execute the command <tt>nltk.download()</tt>. Select the “All Packages” tab and use it to download <tt>averaged_perceptron_tagger</tt>, <tt>maxent_treebank_pos_tagger</tt>, <tt>punkt</tt>, and <tt>wordnet</tt>. If at any point in this notebook you get a LookupError like “Resource 'XXXXX' not found. Please use the NLTK Downloader to obtain the resource”, find the resource in the downloader “All Packages” tab, download it, and try again.

In [1]:
import nltk

In [2]:
#nltk.download()

2.Use the instructions from [3.1](http://www.nltk.org/book/ch03.html) of the NLTK book to have Python download the text of [The Mysterious Affair at Styles](http://www.gutenberg.org/files/863/863-0.txt) from Project Gutenberg. In Python 2.7, the <tt>urlopen</tt> function is located directly in the <tt>urllib</tt> package; not <tt>urllib.request</tt>. Be sure to call the decode <tt>(‘utf8’)</tt> (as in the example) after reading from the URL; this saves the data as a Unicode string (hence you may see a u before every string), which NLTK understands better than the raw data.

In [3]:
import urllib
url = "http://www.gutenberg.org/files/863/863-0.txt"
response = urllib.urlopen(url)
raw = response.read().decode('utf8')

In [4]:
print(len(raw))
print(type(raw))

349459
<type 'unicode'>


3.There’s some text before the book begins and after it ends that we don’t want to include in the analysis. Chapter [3.1](http://www.nltk.org/book/ch03.html) of the NLTK book suggests that you use the string’s <tt>find</tt> and/or <tt>rfind</tt> methods to look for phrases that mark the beginning and ending of the text. 

Find the index corresponding with the beginning of the last (not first, as suggested in the example) occurrence of the string “CHAPTER I. I GO TO STYLES”, and find the index corresponding with the end of the last occurrence of the string “THE END”. Use these indices to discard data before and after these markers. Print the beginning and end of the leftover string to confirm correctness.


In [27]:
i0 = raw.rfind("CHAPTER I. I GO TO STYLES")
print type(i0)
print raw[i0:i0+10]

<type 'int'>
CHAPTER I.


In [6]:
i1 = raw.rfind("THE END")
print raw[i1:i1+7]
i1 = i1+6 # this is really the index we want
print raw[i1:i1-7:-1]

THE END
DNE EHT


In [7]:
text = raw[i0:i1+1]

In [8]:
print text[0:10]
print text[-10:]

CHAPTER I.


THE END


4.Use NLTK's <tt>FreqDist</tt> function to generate a dictionary of the words in the book and the frequency of each. Calculate the “lexical diversity” as defined in [1.4](http://www.nltk.org/book/ch01.html) of the NLTK book, except that you can use the <tt>FreqDist</tt> object you just made rather than a new set to count unique words.

In [9]:
tokens = text.split()
d = nltk.FreqDist(tokens)
lexical_diversity = float(len(d))/len(tokens)   #among all the words in text, how many are unique
print "The ratio of unique tokens to total tokens is:", lexical_diversity

The ratio of unique tokens to total tokens is: 0.176615417316


5.We noticed that the <tt>split()</tt> method left punctuation marks at the end of words, counting these as distinct from the word itself. Improve your frequency distribution dictionary by using NLTK’s <tt>word_tokenize</tt> function instead of <tt>split</tt> to separate the words into a list. Also, before you do that, get rid the original text of underscores (<tt>"__"</tt>), which are used here to indicate emphasis, as these shouldn’t make the enclosed word distinct. 

Calculate the lexical diversity from this FreqDist. 

In [10]:
text = text.replace("_","")
tokens = nltk.word_tokenize(text)
d = nltk.FreqDist(tokens)
lexical_diversity = float(len(d))/len(tokens)
print "The ratio of unique tokens to total tokens is:", lexical_diversity

The ratio of unique tokens to total tokens is: 0.0919265366274


6.It doesn’t really make sense to count “Nature” separately from “nature”, or “Nature” separately from “Natural” or “Naturally” if we’re trying to get a sense of vocabulary. 

Improve your FreqDist by eliminating words that are distinct only in letter case. 

In [11]:
lower = text.lower()
tokens2 = nltk.word_tokenize(lower)
d2 = nltk.FreqDist(tokens)
lexical_diversity = float(len(d2))/len(tokens)
print "The ratio of unique tokens to total tokens is:", lexical_diversity

The ratio of unique tokens to total tokens is: 0.0919265366274


7.Also eliminate those that differ only in affixes (as described in 3.6. You’re welcome to try either of the <tt>stemmers</tt> or the WordNet <tt>lemmatizer</tt> for this. The stemmer is probably more accurate, because the lemmatizer does not reduce all words to their lemma, but the downside is that the stemmers returns <tt>stems</tt> which may not be words at all).

In [28]:
p = nltk.PorterStemmer()
print p.stem("running")
print p.stem("runs")
print p.stem("runned") # The stemmer uses simple rules to try to extract the "stem".
print p.stem("ran")    # It's not perfect because English is so irregular.

<class 'nltk.stem.porter.PorterStemmer'>
run
run
run
ran


In [13]:
stems = map(p.stem,tokens)
d3 = nltk.FreqDist(stems)
lexical_diversity = float(len(d3))/len(tokens)
print "The ratio of unique word stems to total tokens is:", lexical_diversity

The ratio of unique word stems to total tokens is: 0.0677214397931


8.Use the most_common method of the FreqDist from step 6 (not step 7, because that FreqDist might be made up of stems rather than words) to print out the most-used 30 words.

In [14]:
d2.most_common(30)

[(u',', 4280),
 (u'.', 3216),
 (u'the', 2402),
 (u'\u201c', 2228),
 (u'\u201d', 2205),
 (u'I', 1746),
 (u'to', 1314),
 (u'of', 1236),
 (u'a', 1130),
 (u'\u2019', 1020),
 (u'and', 979),
 (u'was', 916),
 (u'that', 883),
 (u'?', 837),
 (u'it', 814),
 (u'in', 752),
 (u'you', 735),
 (u'he', 526),
 (u'is', 520),
 (u'not', 499),
 (u'had', 489),
 (u's', 475),
 (u'her', 443),
 (u'!', 413),
 (u'Poirot', 409),
 (u'his', 392),
 (u'have', 371),
 (u'at', 367),
 (u'me', 362),
 (u'as', 344)]

9.Oops, even after all that, some of the common “words” are actually punctuation marks.  We could remove those from the dictionary, but let’s leave the dictionary alone and instead just show the top 30 words that begin with a letter.

Turn the dictionary-like FreqDist object into a list of word/frequency (key/value) pairs, sort the list by the word frequency numerically (not by word alphabetically) in descending order, and print out the 30 most commonly used words that begin with a letter and their frequency.

In [15]:
l = [(d2[key],key) for key in d2 if key[0].isalpha()]
l = sorted(l, reverse = True)
for pair in l[:30]:
    print pair[1], ":", pair[0]

the : 2402
I : 1746
to : 1314
of : 1236
a : 1130
and : 979
was : 916
that : 883
it : 814
in : 752
you : 735
he : 526
is : 520
not : 499
had : 489
s : 475
her : 443
Poirot : 409
his : 392
have : 371
at : 367
me : 362
as : 344
on : 334
she : 333
my : 328
Inglethorp : 318
with : 315
for : 305
be : 301


10.Some words like “point” are used in several different senses of the word, and intuitively we should somehow factor this into the concept of lexical diversity. Turn the <tt>word_tokenized</tt> list from step 6 into an <tt>nltk.Text</tt> object (3.1) and use the concordance function (1.3) on “point” to see what I mean.

In [16]:
text = nltk.Text(tokens2)

In [17]:
text.concordance("point")

Displaying 20 of 20 matches:
n , stared fixedly over my head at a point on the further wall . it was as thou
 natural faculties . “ and the sixth point ? ” i asked . “ i suppose it is the 
six , but i did not . no , the sixth point i will keep to myself for the presen
 of the tragedy . poirot came to the point at once , with a business-like brisk
” he said . “ do your investigations point to my mother having died a natural d
here , it would have been a valuable point in his favour . that is all. ” john 
nglish say ! you have chosen the one point that to my mind tells against him. ”
he whole riddle ! ” “ and the second point ? ” i asked . “ the important fact t
a judicious mixture of both . on one point , poirot seemed to have a curious ob
ernoon . dorcas ’ s evidence on this point was substantially what poirot and i 
 we need trouble you further on that point . we know all that can be known of t
k ? the coroner went straight to the point . “ on monday evening last , did you
however , p

11.[WordNet](http://www.nltk.org/howto/wordnet.html) maintains a structure of “synsets”, which are like distinct concepts or meanings of words. It would be nice if we could determine the WordNet <tt>synset</tt> with which each occurrence of the word corresponds; that way we could consider the different WordNet <tt>synset</tt> in our lexical diversity calculation. The [Lesk algorithm](http://www.nltk.org/howto/wsd.html) attempts to do just that. 

(a) Print all the different WordNet <tt>synset</tt> definitions for the word “point”.


In [18]:
from nltk.corpus import wordnet as wn

In [19]:
chosen_word = "point"
text = nltk.Text(tokens2)
text.concordance(chosen_word)

Displaying 20 of 20 matches:
n , stared fixedly over my head at a point on the further wall . it was as thou
 natural faculties . “ and the sixth point ? ” i asked . “ i suppose it is the 
six , but i did not . no , the sixth point i will keep to myself for the presen
 of the tragedy . poirot came to the point at once , with a business-like brisk
” he said . “ do your investigations point to my mother having died a natural d
here , it would have been a valuable point in his favour . that is all. ” john 
nglish say ! you have chosen the one point that to my mind tells against him. ”
he whole riddle ! ” “ and the second point ? ” i asked . “ the important fact t
a judicious mixture of both . on one point , poirot seemed to have a curious ob
ernoon . dorcas ’ s evidence on this point was substantially what poirot and i 
 we need trouble you further on that point . we know all that can be known of t
k ? the coroner went straight to the point . “ on monday evening last , did you
however , p

In [20]:
x = wn.synsets(chosen_word)
for i in x:
    print(i, i.definition())

(Synset('point.n.01'), u'a geometric element that has position but no extension')
(Synset('point.n.02'), u'the precise location of something; a spatially limited location')
(Synset('point.n.03'), u'a brief version of the essential meaning of something')
(Synset('detail.n.01'), u'an isolated fact that is considered separately from the whole')
(Synset('degree.n.02'), u'a specific identifiable position in a continuum or series or especially in a process')
(Synset('point.n.06'), u'an instant of time')
(Synset('point.n.07'), u'the object of an activity')
(Synset('point.n.08'), u'a V shape')
(Synset('point.n.09'), u'a very small circular shape')
(Synset('point.n.10'), u'the unit of counting in scoring a game or contest')
(Synset('point.n.11'), u'a promontory extending out into a large body of water')
(Synset('item.n.01'), u'a distinct part that can be specified separately in a group of things that could be enumerated on a list')
(Synset('point.n.13'), u'a style in speech or writing that arre

(b) The Lesk algorithm needs the sentence the word appears in to determine the corresponding synset. 

Write a function that returns a list of all the sentences in the original text in which the word appears. 

Print all the sentences in which the word “point” appears. [Chapter 3.8](http://www.nltk.org/book/ch03.html) of the book will be useful.

In [21]:
def get_sentences(word,text):
    sentences = nltk.sent_tokenize(text)
    return [s for s in sentences if word in nltk.word_tokenize(s)]

text = raw[i0:i1+1]
sentences = get_sentences(chosen_word,text)
for sentence in sentences:
    print sentence

print len(sentences)

He
was white as chalk, the candle he held in his shaking hand was
sputtering onto the carpet, and his eyes, petrified with terror, or
some such kindred emotion, stared fixedly over my head at a point on
the further wall.
“And the sixth point?” I asked.
No, the sixth point I will keep to myself for the
present.”

He looked quickly round the room.
Poirot came to the point at once, with a business-like briskness.
“Do your investigations
point to my mother having died a natural death—or—or must we prepare
ourselves for the worst?”

“I think, Mr. Cavendish,” said Poirot gravely, “that you would do well
not to buoy yourself up with any false hopes.
If anyone had chanced to look this morning before his
return, and seen it there, it would have been a valuable point in his
favour.
You have chosen the one point that to
my mind tells against him.”

“How is that?”

“Because if Mr. Inglethorp knew that his wife would be poisoned last
night, he would certainly have arranged to be away from the house

(c) Use the <tt>lesk</tt> function to determine the synset definition and part of speech of “point” in each sentence. (It does very poorly.)

Write a function that attempts to disambiguate the sense of the word in 
a sentence using the Lesk algorithm, then return the definition of that
sense of the word.

In [22]:
from nltk.wsd import lesk

In [23]:
def my_lesk(word,sentence):
    syn = lesk(nltk.word_tokenize(sentence),word)
    return syn.definition(),syn.pos()

In [24]:
for sentence in sentences:
    print sentence
    definition,pos = my_lesk(chosen_word,sentence)
    print "The definition of point, as used in this sentence, is:"
    print definition
    print "\n"

He
was white as chalk, the candle he held in his shaking hand was
sputtering onto the carpet, and his eyes, petrified with terror, or
some such kindred emotion, stared fixedly over my head at a point on
the further wall.
The definition of point, as used in this sentence, is:
one percent of the total principal of a loan; it is paid at the time the loan is made and is independent of the interest on the loan


“And the sixth point?” I asked.
The definition of point, as used in this sentence, is:
direct the course; determine the direction of travelling


No, the sixth point I will keep to myself for the
present.”

He looked quickly round the room.
The definition of point, as used in this sentence, is:
give a point to


Poirot came to the point at once, with a business-like briskness.
The definition of point, as used in this sentence, is:
a punctuation mark (.) placed at the end of a declarative sentence to indicate a full stop or after abbreviations


“Do your investigations
point to my mo

(d) Maybe if we tag the parts of speech first using the <tt>pos_tag</tt> function [(Chapter 5)](http://www.nltk.org/book/ch05.html) and provide that information as the (optional) second argument to the <tt>lesk</tt> function, it will do a better job?

Note that <tt>pos_tag</tt> doesn’t provide the part of speech for just a single word, and you’ll have to convert between the output provided by the part of speech tagger and a single letter representing the part of speech required by <tt>lesk</tt>. This doesn’t have to work for all possible parts of speech, just the parts of speech of “point” in the text.
This still does poorly, but it was a good idea. It seems like the <tt>Lesk</tt> algorithm just tries to match up words from the context sentence with words from the synset definition, which is probably too short for this approach to work well. There’s certainly a lot of work left to be done in the field of natural language processing!

In [25]:
def my_lesk2(word,sentence):
    sen_tokens = nltk.word_tokenize(sentence)
    sen_tagged = nltk.pos_tag(sen_tokens)
    ind = sen_tokens.index(word)
    pos = sen_tagged[ind][1]
    syn = lesk(sen_tokens,word,pos= pos[0].lower())
    return syn.definition(),pos

In [26]:
for sentence in get_sentences(chosen_word, raw):
    print "Word is:",chosen_word    
    print "As in:", sentence
    meaning, pos =  my_lesk(chosen_word,sentence)
    print "Without POS:", meaning
    print "Assumed POS:", pos
    meaning, pos =  my_lesk2(chosen_word,sentence)
    print "With POS:",meaning
    print "POS:",pos
    print ""

Word is: point
As in: He
was white as chalk, the candle he held in his shaking hand was
sputtering onto the carpet, and his eyes, petrified with terror, or
some such kindred emotion, stared fixedly over my head at a point on
the further wall.
Without POS: one percent of the total principal of a loan; it is paid at the time the loan is made and is independent of the interest on the loan
Assumed POS: n
With POS: one percent of the total principal of a loan; it is paid at the time the loan is made and is independent of the interest on the loan
POS: NN

Word is: point
As in: “And the sixth point?” I asked.
Without POS: direct the course; determine the direction of travelling
Assumed POS: v
With POS: the gun muzzle's direction
POS: NN

Word is: point
As in: No, the sixth point I will keep to myself for the
present.”

He looked quickly round the room.
Without POS: give a point to
Assumed POS: v
With POS: the property of a shape that tapers to a sharp tip
POS: NN

Word is: point
As in: Poirot